# 31. Auditoria: Agente de Análise de Diário Oficial (PDF)

Este notebook demonstra como criar um "Agente Auditor" que:
1.  Lê um arquivo PDF de Diário Oficial (upload do usuário).
2.  Extrai estruturadamente todas as compras e contratações usando **Google Gemini** e **Pydantic**.
3.  Analisa os dados extraídos em busca de anomalias ou pontos de atenção.
4.  Gera um Relatório Executivo automático.

**Fluxo:** PDF -> Texto -> Extração Estruturada (LLM) -> Análise de Regras -> Relatório Final.

In [1]:
### INJECTION START ###
import os
from dotenv import load_dotenv
# Carregar variáveis de ambiente locais se existirem
load_dotenv()
### INJECTION END ###

# Instalação das dependências
!pip install -qU langchain langchain-google-genai pypdf


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


## 1. Carregando o Diário Oficial (Upload)

Faça o upload do arquivo PDF do Diário Oficial que deseja auditar.

In [2]:
import os
try:
    from google.colab import files
    print("Por favor, faça o upload do seu arquivo PDF de Diário Oficial.")
    uploaded = files.upload()
    if uploaded:
        pdf_filename = next(iter(uploaded))
        print(f"Arquivo '{pdf_filename}' carregado com sucesso!")
    else:
        print("Nenhum arquivo enviado.")
        pdf_filename = None
except ImportError:
    print("Ambiente local detectado.")
    # Forçar uso do arquivo específico se solicitado ou listar pdfs
    target_pdf = "diario-oficial-ordinaria-5403-02-04-2026-19_18_50.pdf"
    
    if os.path.exists(target_pdf):
        pdf_filename = target_pdf
        print(f"Usando arquivo solicitado: {pdf_filename}")
    else:
        # Verifica se existe algum PDF na pasta
        pdfs = [f for f in os.listdir('.') if f.lower().endswith('.pdf')]
        if pdfs:
            print(f"PDFs encontrados na pasta: {pdfs}")
            pdf_filename = pdfs[0]
            print(f"Usando automaticamente o primeiro PDF encontrado: {pdf_filename}")
        else:
            pdf_filename = input("Digite o caminho/nome do arquivo PDF (ex: diario.pdf): ")
            if not os.path.exists(pdf_filename):
                 print("Arquivo não encontrado. Certifique-se de que ele está na pasta.")


Ambiente local detectado.
Usando arquivo solicitado: diario-oficial-ordinaria-5403-02-04-2026-19_18_50.pdf


## 2. Carregando e Processando o PDF

Uso do `PyPDFLoader` para obter o texto bruto.

In [3]:
from langchain_community.document_loaders import PyPDFLoader

if pdf_filename:
    loader = PyPDFLoader(pdf_filename)
    pages = loader.load()
    full_text = "\n".join([p.page_content for p in pages])
    
    print(f"Texto carregado ({len(full_text)} caracteres).")
    # print(full_text) # Opcional: imprimir texto para conferir
else:
    print("ERRO: Nenhum arquivo PDF definido.")

Texto carregado (429648 caracteres).


## 3. Definindo o Schema de Extração (Pydantic)

Definimos exatamente quais dados queremos capturar de cada compra.

In [4]:
from typing import List, Optional
from pydantic import BaseModel, Field

class Compra(BaseModel):
    """Representa uma compra ou contrato extraído do diário oficial."""
    numero_contrato: Optional[str] = Field(None, description="Número do contrato ou processo")
    contratante: Optional[str] = Field(None, description="Órgão ou secretaria que está comprando")
    contratada: Optional[str] = Field(None, description="Nome da empresa ou pessoa contratada")
    objeto: Optional[str] = Field(None, description="Descrição do bem ou serviço adquirido")
    valor: Optional[float] = Field(None, description="Valor total da compra em Reais (converter para float)")
    modalidade: Optional[str] = Field(None, description="Modalidade (Licitação, Dispensa, Pregão, etc)")

class DiarioOficial(BaseModel):
    """Lista de todas as compras encontradas no texto."""
    compras: List[Compra] = Field(description="Lista de compras e contratos identificados")

## 4. Agente de Extração (Gemini)

Usamos `with_structured_output` para obrigar o LLM a retornar JSON conforme nosso schema.

In [5]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

# Configuração da API
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = "SUA_CHAVE_AQUI" # No Colab use userdata
    try:
        from google.colab import userdata
        os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    except:
        pass

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

structured_llm = llm.with_structured_output(DiarioOficial)

prompt = ChatPromptTemplate.from_template(
    """Você é um assistente de IA especializado em ler Diários Oficiais.
    Extraia TODAS as informações sobre compras, contratos e licitações do texto abaixo.
    Se não encontrar o valor exato, tente estimar ou extrair o número disponível.
    Ignore nomeações de pessoal ou decretos que não envolvam gastos com fornecedores.
    
    TEXTO DO DIÁRIO:
    {text}
    """
)

chain_extract = prompt | structured_llm

In [6]:
if pdf_filename:
    print("⏳ Extraindo dados com IA...")
    resultado = chain_extract.invoke({"text": full_text})
    
    print(f"✅ Extração completa! Encontradas {len(resultado.compras)} compras.")
    for i, compra in enumerate(resultado.compras):
        valor_fmt = f"R$ {compra.valor:,.2f}" if compra.valor is not None else "N/A"
        contratada_fmt = compra.contratada if compra.contratada else "N/A"
        objeto_fmt = f"{compra.objeto[:50]}..." if compra.objeto else "N/A"
        print(f"{i+1}. {contratada_fmt} | {valor_fmt} | {objeto_fmt}")

⏳ Extraindo dados com IA...


✅ Extração completa! Encontradas 99 compras.
1. 62.535.481 CAMILA ALVES DA CRUZ | R$ 8,025.00 | Aquisição de gêneros alimentícios não perecíveis e...
2. AJR COMÉRCIO DE PRODUTOS PARA SAÚDE LTDA | R$ 125,000.00 | Aquisição de gêneros alimentícios não perecíveis e...
3. AM COMERCIO DE ALIMENTOS LTDA | R$ 8,970.00 | Aquisição de gêneros alimentícios não perecíveis e...
4. COMERCIAL BRAZ LTDA | R$ 58,200.00 | Aquisição de gêneros alimentícios não perecíveis e...
5. ELLEV DISTRIBUIDORA LTDA | R$ 982,589.30 | Aquisição de gêneros alimentícios não perecíveis e...
6. J L DALLAGNOL EMPREENDIMENTOS IMOBILIARIOS LTDA | R$ 5,648.37 | N/A
7. JG ARQUITETURA LTDA | R$ 3,911.27 | N/A
8. ROSA MARIA JERONYMO LIMA | R$ 14,239.37 | N/A
9. ANDRE DALLEASTE DOS SANTOS | R$ 15,976.08 | N/A
10. MODERNATIVA EMPREENDIMENTOS IMOBILIÁRIOS LTDA | R$ 3,548.28 | N/A
11. BEATRIZ APARECIDA DA SILVA MOSQUER | R$ 35,542.50 | N/A
12. ANB ENGENHARIA LTDA | R$ 4,818.70 | N/A
13. ANB ENGENHARIA LTDA | R$ 3,881.49 | N/A
14. O

## 5. Agente de Auditoria (Análise)

Agora vamos criar uma função que avalia as compras extraídas e busca "red flags" (bandeiras vermelhas).

In [7]:
def analisar_compras(dados: DiarioOficial) -> str:
    analise_prompt = ChatPromptTemplate.from_template(
        """Você é um Auditor Geral. Analise a lista de compras extraídas do Diário Oficial abaixo.
        Identifique possíveis irregularidades, preços abusivos ou descrições vagas. 
        Dê atenção especial a compras de alto valor ou itens de luxo/desnecessários.
        
        DADOS DAS COMPRAS:
        {dados_json}
        
        Gere um RELATÓRIO EXECUTIVO em Markdown contendo:
        1. Resumo dos gastos (Total, Maior Compra).
        2. Pontos de Atenção (Red Flags) - com justificativa.
        3. Recomendações de Fiscalização.
        """
    )
    
    # Convertendo os dados Pydantic para JSON string para o prompt
    dados_json = dados.json()
    
    chain_analise = analise_prompt | llm
    response = chain_analise.invoke({"dados_json": dados_json})
    return response.content

if pdf_filename:
    print("📊 Gerando Relatório de Auditoria...")
    relatorio = analisar_compras(resultado)
    from IPython.display import Markdown
    display(Markdown(relatorio))

📊 Gerando Relatório de Auditoria...


/var/folders/kg/kfw4qstx31j1zmw50lzf4mn80000gn/T/ipykernel_91287/3599643814.py:18: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  dados_json = dados.json()


## RELATÓRIO EXECUTIVO DE AUDITORIA - COMPRAS MUNICÍPIO DE FOZ DO IGUAÇU - PR

**Data:** 2024-01-24

**Escopo:** Análise de lista de compras extraídas do Diário Oficial do Município de Foz do Iguaçu - PR.

### 1. Resumo dos Gastos

*   **Total de Gastos Identificados:** R$ 7.484.248,84
*   **Maior Compra:** R$ 2.247.397,48 - Prestação de serviço de forma contínua, dos serviços de manutenção preventiva e corretiva por Veículos Pesados, Caminhão, Ônibus, Equipamentos Rodoviários e Agrícola, incluído mecânica, elétrica, lanternagem, pintura, higienização completa, retífica de motores elétricos, combustão, equipamentos acoplados sendo; Placas Vibratórias, Sopradores, e Consertos de Ferramentas Elétricas, serviços de tapeçaria, serviços de guincho, serviços de manutenção tipo mão-de-obra, conserto com reposição, incluindo o fornecimento de peças, multimídia e acessórios de reposição genuínas e/ou originais, para atender às necessidades da frota de veículos sendo: Secretaria Municipal de Obras, Secretaria Municipal de Meio Ambiente, Secretaria Municipal de Educação, Secretaria Municipal da Agricultura e Secretaria Municipal de Assistência Social. Contratada: RETIFICADORA DE MOTORES FOZ LTDA.

### 2. Pontos de Atenção (Red Flags)

*   **Objeto Vago e Repetitivo:** Diversas compras da Secretaria Municipal de Educação (representando um volume financeiro significativo) possuem o mesmo objeto: "Aquisição de gêneros alimentícios não perecíveis e insumos essenciais ao funcionamento e à manutenção das unidades escolares, bem como ao atendimento administrativo e institucional da Secretaria Municipal de Educação".
    *   **Justificativa:** A descrição genérica dificulta a identificação dos itens específicos adquiridos, impedindo a análise de preços unitários e a comparação com valores de mercado. A repetição do objeto sugere a possibilidade de fracionamento indevido de despesas para evitar modalidades de licitação mais rigorosas.
*   **Valor Elevado em Manutenção de Veículos:** O contrato de R$ 2.247.397,48 com a RETIFICADORA DE MOTORES FOZ LTDA para manutenção de veículos pesados e equipamentos é o maior valor identificado.
    *   **Justificativa:** O valor elevado exige uma análise detalhada dos serviços prestados, da necessidade real de manutenção, da compatibilidade dos preços praticados com o mercado e da qualidade das peças e serviços fornecidos. A abrangência do contrato (diversas secretarias) demanda um controle rigoroso da execução.
*   **Contratação de Artista com Data Futura:** A contratação da banda Falamansa para o Carnaval de 2026, com um valor de R$ 220.000,00, chama a atenção pela data futura do evento.
    *   **Justificativa:** Embora a contratação de artistas seja comum, a antecipação de quase dois anos para o evento levanta questionamentos sobre a necessidade premente da contratação, a possibilidade de mudanças nas condições do mercado e a garantia da efetiva realização do evento.
*   **Inexigibilidade para Manutenção de Equipamento Específico:** A contratação por inexigibilidade da MEDICALBLU EQUIPAMENTOS MÉDICOS E HOSPITALARES LTDA para manutenção de torre de vídeo Stryker (R$ 86.870,86).
    *   **Justificativa:** É necessário verificar se a inexigibilidade foi devidamente justificada, comprovando a exclusividade da empresa na prestação do serviço e a impossibilidade de competição. É importante analisar se o preço cobrado é compatível com o mercado, mesmo em situação de inexigibilidade.
*   **Ausência de Informações:** Um bloco de transações não possui informações sobre o contratante, contratada, objeto e modalidade.
    *   **Justificativa:** A ausência de informações impede qualquer análise e levanta sérias dúvidas sobre a legalidade e a transparência dessas transações. É imprescindível identificar a origem e a natureza desses gastos.

### 3. Recomendações de Fiscalização

1.  **Auditoria Detalhada das Compras de Gêneros Alimentícios:**
    *   Levantar os itens específicos adquiridos em cada compra da Secretaria Municipal de Educação.
    *   Comparar os preços unitários com os valores de mercado, buscando identificar possíveis sobrepreços.
    *   Analisar a necessidade e a adequação dos itens adquiridos para as unidades escolares.
    *   Verificar se houve fracionamento indevido de despesas.
2.  **Inspeção da Execução do Contrato de Manutenção de Veículos:**
    *   Analisar os relatórios de manutenção, verificando os serviços prestados, as peças substituídas e os custos envolvidos.
    *   Realizar inspeções *in loco* nos veículos e equipamentos para verificar a qualidade da manutenção.
    *   Comparar os preços praticados pela RETIFICADORA DE MOTORES FOZ LTDA com os valores de mercado.
    *   Verificar se a distribuição dos serviços entre as secretarias está sendo realizada de forma eficiente e transparente.
3.  **Análise da Legalidade da Contratação da Banda Falamansa:**
    *   Verificar a justificativa para a antecipação da contratação.
    *   Analisar a compatibilidade do preço contratado com os valores praticados no mercado.
    *   Garantir a existência de cláusulas contratuais que protejam o município em caso de cancelamento do evento.
4.  **Verificação da Inexigibilidade na Manutenção da Torre de Vídeo:**
    *   Analisar a documentação que justifica a inexigibilidade, comprovando a exclusividade da MEDICALBLU EQUIPAMENTOS MÉDICOS E HOSPITALARES LTDA.
    *   Comparar o preço cobrado com os valores praticados no mercado para serviços similares.
5.  **Identificação e Análise das Transações Sem Informação:**
    *   Realizar uma investigação completa para identificar a origem e a natureza das transações que não possuem informações.
    *   Verificar a legalidade e a regularidade desses gastos.
6.  **Análise dos Processos de Credenciamento:**
    *   Verificar se os processos de credenciamento público para educadores artístico-culturais estão seguindo os princípios da legalidade, impessoalidade, moralidade, publicidade e eficiência.
    *   Analisar os critérios de seleção e a qualificação dos educadores contratados.

**Conclusão:**

A análise preliminar da lista de compras do Município de Foz do Iguaçu - PR revela diversos pontos de atenção que exigem uma fiscalização mais aprofundada. As recomendações apresentadas visam garantir a correta aplicação dos recursos públicos, a transparência dos gastos e a proteção do patrimônio municipal.

### Conclusão

Criamos um pipeline completo 'End-to-End':
1.  O PDF entra no sistema.
2.  O Gemini estrutura os dados não estruturados.
3.  O Gemini (em outra chamada) atua como auditor e gera inteligência sobre os dados.